In [13]:
from itertools import chain

def generate_cb(i, valid = True):
    import json
    o = {
        "_label_cost": - (i % 2),
        "_label_probability": 0.5,
        "_label_Action": i % 2 + 1,
        "_labelIndex": i % 2,
        "a": [i % 2, (i + 1) % 2],
        "c": {"f": 1, '_multi': [{'f': 1}, {'f': 2}]},
        "p": [0.5, 0.5] 
    }
    return f'{json.dumps(o, separators=(",", ":"))}{"" if valid else "!"}\n'

def generate_file(generator, lines, path):
    with open(path, 'w') as f:
        f.writelines(map(lambda i: generator(i), range(lines)))

def generate_file_with_invalid(lines, path):
    with open(path, 'w') as f:
        for i in range(lines):
            f.write(generate_cb(i))
        f.write(generate_cb(0, False))
        for i in range(lines):
            f.write(generate_cb(i))          


In [14]:
generate_file(generate_cb, 10, 'vw_executor/tests/data/cb_0.json')
generate_file(generate_cb, 11, 'vw_executor/tests/data/cb_1.json')
generate_file_with_invalid(10, 'vw_executor/tests/data/cb_invalid.json')

In [17]:
from vw_executor.vw import Vw
from vw_executor.vw_opts import dimension, product
import pandas as pd

vw = Vw('vw', '.vw_cache')

inputs = [
    'vw_executor/tests/data/cb_0.json',
    'vw_executor/tests/data/cb_1.json',
    'vw_executor/tests/data/cb_invalid.json'
    ]

opts = pd.DataFrame(product(
    dimension('#base', ['--ccb_explore_adf -P 5 --preserve_performance_counters --save_resume --dsjson']),
    dimension('--cb_type', ['ips']),
))


In [18]:
result = vw._with(reset=True).train(inputs, opts)
result

Total:   0%|          | 0/1 [00:00<?, ?it/s]

--cb_type ips:   0%|          | 0/3 [00:00<?, ?it/s]

ValueError: All arrays must be of the same length

In [16]:
result.iloc[0]['!Job'][-1].loss_table

,loss,since_last
i,,
